In [2]:
import numpy as np 
import pandas as pd
import os
import io
import glob
import re
import sys
from datetime import datetime, timedelta

import plotnine
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import folium
from folium import plugins
import branca
import json

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['SeoulFloating',
 'TimeAge',
 'SearchTrend',
 'TimeProvince',
 'Weather',
 'PatientRoute',
 'PatientInfo',
 'Region',
 'TimeGender',
 'Case',
 'Time']

## 일별 누적 확진자수 그래프

In [4]:
timedat = Time.set_index('date').drop('time',axis=1)

In [5]:
fig = timedat.confirmed.iplot(asFigure=True, kind='scatter')
fig.show()

## 일별 확진자수 그래프

In [65]:
dailyconf = timedat.confirmed.diff()
dailyconf[0] = 1

In [7]:
fig1 = dailyconf.iplot(asFigure=True, kind='scatter')
fig1.show()

## 주별 데이터로 바꾸기

In [66]:
dailyconf = pd.DataFrame(dailyconf)
dailyconf.index = pd.to_datetime(dailyconf.index)
dailyconf['week'] = dailyconf.index.week
dailyconf['year'] = pd.DatetimeIndex(Time.date).year

In [72]:
weekconf = dailyconf.groupby(['year','week']).sum() 

In [80]:
fig2 = weekconf.iplot(asFigure=True, kind='scatter')
fig2.show()

## 호흡기 세포 융합 바이러스(RSV) 

* 잠복기 평균 5일
* 전파경로: 감염자의 분비물과 직접 접촉 혹은 비말 전파
* 바이러스는 증상이 나타나기 수일 전부터 배출 가능. 
* 고령자나 면역저하자에게 입원을 요하는 폐렴의 원인
* 인플루엔자와 비슷한 빈도로 발생
* 10월 - 3월 사이에 주로 발생

In [115]:
rsv = pd.read_csv('rsv.csv',header=None)
rsv = rsv.set_index([0,1])
rsv = rsv.drop(3,axis=1)
rsv.columns = ['rsv']

In [116]:
fig3 = rsv.iplot(asFigure=True, kind='scatter')
fig3.show()

모양은 상당히 코로나랑 비슷한 것 같다.

## 코로나 데이터와 함께 그려보자.

In [117]:
comb = pd.concat([rsv,weekconf],axis=1)
comb.columns = ['rsv','corona']

In [119]:
fig4 = comb.iplot(asFigure=True,kind='scatter')
fig4.show()

비슷한 경로로 전염되는 질병인 rsv 보다 전파 속도가 훨씬 빨랐고, 한편 감염자가 줄어드는 속도도 빨랐다.